In [1]:
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
#Importing the categorised data
data = pd.read_excel("WithCat.xlsx")
data

,Category,Status,Company,Contact Person,Designation,Decision maker,Email ID,Phone,Notes prospects (for sentiment analisys),Country,Website,Address,Notes sales process,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,CB,Hot lead,Barcelona Convention Bureau,Monica Colomer,Trade fairs & web manager,no,BCB@BARCELONATURISME.COM,0034 3689700,They have another person who is already going ...,SPAIN,http://www.barcelonaturisme.com,NaN,They have another person who is already going ...,NaN,NaN,NaN
2,CB,Hot lead,Belgium - Visit.Brussels Convention Bureau,Thierry Van Eyll,NaN,no,SALES@VISIT.BRUSSELS,0032 25495052,NaN,BELGIUM,NaN,RUE ROYALE 2-4 KONINGSSTRAAT BRUSSELS BRUSSELS...,said to talk to Pieter,NaN,NaN,NaN
5,CB,No response,Cologne Convention Bureau – Kölntourismus Gmbh,Christian Woronka,Head of bureau,yes,CHRISTIAN.WORONKA@KOELNTOURISMUS.DE,0049 22134643210,NaN,GERMANY,NaN,KARDINAL-HÖFFNER-PLATZ 1 KÖLN NORTH RHINE-WEST...,"didn't pick up 6 times,",NaN,NaN,NaN
6,CB,Hot lead,Convention Bureau Düsseldorf,Julia Hack,NaN,yes,INFO@CONVENTION-DUESSELDORF.COM,0049 21117202858,NaN,GERMANY,NaN,"MARKTPLATZ 6 DÜSSELDORF 40213 ,GERMANY",talked to another Julia... Julia Hack is out o...,NaN,NaN,NaN
7,CB,Not interested,Convention Bureau Lower Austria,Edith Mader,NaN,yes,CONVENTION@NOE.CO.AT,0043 2742900019825,NaN,AUSTRIA,NaN,"NIEDERÖSTERREICH-RING 2, HAUS C ST. PÖLTEN AUS...",they are too small and operate only in Austria...,NaN,NaN,NaN
8,CB,Not interested,Convention Bureau Upper Austria,Andreas,NaN,no,tagung@oberoesterreich.at,0043 732 7277581,NaN,Austria,www.oberoesterreich.at/tagungen,Freistaedter Strasse 119 AT-4041 Linz,"talked to Andreas, said to email or call ms. I...",NaN,NaN,NaN
10,CB,Not interested,Convention Edinburgh,Mrs. Lesley Williams,Head of Business Tourism,yes,conferences@conventionedinburgh.com,0044 131 4733666,NaN,United Kingdom,www.conventionedinburgh.com/,26 Frederick Street GB- Edinburgh EH2 2JR,"not interested, failed the call",NaN,NaN,NaN
11,TB,Hot lead,Croatian National Tourist Board,Ms. Ivana Karuza,Expert Associate,no,meetings@htz.hr,00385 1 4699 331,"They are working with Germany, Europe, United ...",Croatia,www.croatia.hr/,Iblerov trg 10/IV HR-10 000 Zagreb,"she is not decision makes, but she can forward...",NaN,NaN,NaN
14,CB,Customer,Cyprus Tourism Organisation - Cyprus Conventio...,reception,NaN,NaN,-,00357 22691100,-,Cyprus,NaN,"P.O. Box 24535, Nicosia, 1390, CYPRUS",-,NaN,NaN,NaN
17,CB,Cold lead,Czechtourism - Czech Convention Bureau,Zdeněk Giormani,NaN,no,info@czechtourism.cz/ giormani@czechtourism.cz,00420 221580111,NaN,CZECH REPUBLIC,NaN,VINOHRADSKA 46 PO BOX 32 PRAGUE 2 CZ-120 41 PR...,"he doesn't know whom to connect me with, so I ...",NaN,NaN,NaN


In [3]:
# Splitting the data into features and labels 
fdata = data.loc[0:933,['Company']]
ldata = data.loc[0:933,['Category']]

In [4]:
print(fdata)
print(ldata)

                                               Company
0                          Barcelona Convention Bureau
2           Belgium - Visit.Brussels Convention Bureau
5       Cologne Convention Bureau – Kölntourismus Gmbh
6                         Convention Bureau Düsseldorf
7                      Convention Bureau Lower Austria
8                      Convention Bureau Upper Austria
10                                Convention Edinburgh
11                     Croatian National Tourist Board
14   Cyprus Tourism Organisation - Cyprus Conventio...
17              Czechtourism - Czech Convention Bureau
18                       La Cité des Congrès de Nantes
19                            Dublin Convention Bureau
21                         Edinburgh Conference Centre
23                           Finland Convention Bureau
26                         Frankfurt Convention Bureau
28            Georgia Convention And Exhibition Bureau
30                      German Convention Bureau (Gcb)
32        

In [5]:
#Cleaning the Comapny names - Getting rid of the punctuation marks, etc.
fdata['Company'] = fdata['Company'].str.replace('[{}]'.format(string.punctuation), '')
fdata['Company'] = fdata['Company'].str.replace('-', '')
fdata['Company'] = fdata['Company'].str.replace('.', '')
fdata['Company'] = fdata['Company'].str.replace(' - ', '')

In [6]:
fdata

,Company
0,Barcelona Convention Bureau
2,Belgium VisitBrussels Convention Bureau
5,Cologne Convention Bureau – Kölntourismus Gmbh
6,Convention Bureau Düsseldorf
7,Convention Bureau Lower Austria
8,Convention Bureau Upper Austria
10,Convention Edinburgh
11,Croatian National Tourist Board
14,Cyprus Tourism Organisation Cyprus Convention...
17,Czechtourism Czech Convention Bureau


In [7]:
# Tokenizing the feature data, i.e.,the company names and then forming sequences.
vocabulary_size = 5000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(fdata['Company'])
sequences = tokenizer.texts_to_sequences(fdata['Company'])
data = pad_sequences(sequences, maxlen=6)

In [8]:
#The LSTM model.
model = Sequential()
model.add(Embedding(5000, 100, input_length=6))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(11, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
#Cleaning the Category data.
ldata['Category'] = ldata['Category'].str.replace('Congress center', 'CC')
ldata['Category'] = ldata['Category'].str.replace('Conference center', 'CC')
ldata.loc[ldata.Category == 'T', 'Category'] = 'TA'
ldata

,Category
0,CB
2,CB
5,CB
6,CB
7,CB
8,CB
10,CB
11,TB
14,CB
17,CB


In [10]:
#Encoding the category data for the model.
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
y=ldata["Category"]
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[ 1  1  1  1  1  1  1  8  1  1  2  1  2  1  1  1  1  1  1  1  1  1  9  1
  1  9  8  1  1  1  1  1  1  1  2  1  2  1  1  1  1  1  1  9  9  9  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  9  7  7  7  7  7
  5  5  5  5  3  5  6  5  5  3  3  3  5  5  3  3  5  5  5  5  5  5  5  5
  5  3  3 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10  4 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10  7  7  7  7  7  7  7  7  7 10 10 10 10  7  4 10 10 10 10 10  4 10 10
 10 10 10 10 10 10 10 10 10 10 10 10  6 10 10  6 10  6 10  6 10 10 10 10
  5  5  6  4 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10  4 10 10
 10 10 10 10  4 10 10 10 10 10 10  4 10 10 10 10 10 10 10 10 10  3  3  3
  3  3 10  5  6  6  3  3  5  3  5  6  5  5  5  5  5  5  3  3  5  5  5  6
  5  5  5  5  5  5  3  5  5  5  5 10 10 10 10  5 10 10 10 10 10 10 10 10
 10 10  4  4 10 10 10  9  9  9  9  9  9  9  9  9  9

In [11]:
ldata.Category.unique()

array(['CB', 'TB', 'CC', 'TO', 'TA', 'H', 'CH', 'R', 'WP', 'EP', 'A'],
      dtype=object)

In [12]:
#Fitting the model : This gives an accuracy of 90.2%
model.fit(data, np.array(onehot_encoded), validation_split=0.35, epochs=1000)

Train on 435 samples, validate on 235 samples
Epoch 1/1000
435/435 [==============================] - 11s 26ms/step - loss: 0.6671 - acc: 0.8357 - val_loss: 0.6179 - val_acc: 0.8453
Epoch 2/1000
435/435 [==============================] - 0s 828us/step - loss: 0.4353 - acc: 0.9016 - val_loss: 0.3580 - val_acc: 0.8495
Epoch 3/1000
435/435 [==============================] - 0s 837us/step - loss: 0.2478 - acc: 0.9099 - val_loss: 0.3862 - val_acc: 0.9091
Epoch 4/1000
435/435 [==============================] - 0s 846us/step - loss: 0.2325 - acc: 0.9091 - val_loss: 0.3742 - val_acc: 0.9091
Epoch 5/1000
435/435 [==============================] - 0s 837us/step - loss: 0.2283 - acc: 0.9149 - val_loss: 0.3625 - val_acc: 0.9091
Epoch 6/1000
435/435 [==============================] - 0s 846us/step - loss: 0.2243 - acc: 0.9114 - val_loss: 0.3675 - val_acc: 0.9133
Epoch 7/1000
435/435 [==============================] - 0s 846us/step - loss: 0.2185 - acc: 0.9308 - val_loss: 0.3666 - val_acc: 0.9126
Ep

435/435 [==============================] - 0s 864us/step - loss: 0.0046 - acc: 0.9998 - val_loss: 0.3708 - val_acc: 0.9079
Epoch 61/1000
435/435 [==============================] - 0s 828us/step - loss: 0.0045 - acc: 0.9998 - val_loss: 0.3821 - val_acc: 0.9068
Epoch 62/1000
435/435 [==============================] - 0s 828us/step - loss: 0.0042 - acc: 0.9998 - val_loss: 0.3854 - val_acc: 0.9079
Epoch 63/1000
435/435 [==============================] - 0s 818us/step - loss: 0.0041 - acc: 0.9998 - val_loss: 0.3837 - val_acc: 0.9083
Epoch 64/1000
435/435 [==============================] - 0s 837us/step - loss: 0.0040 - acc: 0.9998 - val_loss: 0.3870 - val_acc: 0.9075
Epoch 65/1000
435/435 [==============================] - 0s 828us/step - loss: 0.0039 - acc: 0.9996 - val_loss: 0.3883 - val_acc: 0.9079
Epoch 66/1000
435/435 [==============================] - 0s 828us/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.3895 - val_acc: 0.9075
Epoch 67/1000
435/435 [==============================] 

435/435 [==============================] - 0s 864us/step - loss: 9.9197e-04 - acc: 1.0000 - val_loss: 0.5039 - val_acc: 0.9068
Epoch 120/1000
435/435 [==============================] - 0s 855us/step - loss: 9.4508e-04 - acc: 1.0000 - val_loss: 0.4992 - val_acc: 0.9068
Epoch 121/1000
435/435 [==============================] - 0s 846us/step - loss: 9.0649e-04 - acc: 1.0000 - val_loss: 0.5090 - val_acc: 0.9064
Epoch 122/1000
435/435 [==============================] - 0s 855us/step - loss: 8.8838e-04 - acc: 1.0000 - val_loss: 0.5113 - val_acc: 0.9060
Epoch 123/1000
435/435 [==============================] - 0s 855us/step - loss: 8.6442e-04 - acc: 1.0000 - val_loss: 0.5122 - val_acc: 0.9064
Epoch 124/1000
435/435 [==============================] - 0s 855us/step - loss: 8.9773e-04 - acc: 1.0000 - val_loss: 0.5088 - val_acc: 0.9064
Epoch 125/1000
435/435 [==============================] - 0s 846us/step - loss: 8.4188e-04 - acc: 1.0000 - val_loss: 0.5149 - val_acc: 0.9064
Epoch 126/1000
435/43

Epoch 177/1000
435/435 [==============================] - 0s 920us/step - loss: 4.3136e-04 - acc: 1.0000 - val_loss: 0.5716 - val_acc: 0.9052
Epoch 178/1000
435/435 [==============================] - 0s 837us/step - loss: 4.1405e-04 - acc: 1.0000 - val_loss: 0.5695 - val_acc: 0.9048
Epoch 179/1000
435/435 [==============================] - 0s 818us/step - loss: 3.8211e-04 - acc: 1.0000 - val_loss: 0.5751 - val_acc: 0.9052
Epoch 180/1000
435/435 [==============================] - 0s 837us/step - loss: 3.7684e-04 - acc: 1.0000 - val_loss: 0.5778 - val_acc: 0.9056
Epoch 181/1000
435/435 [==============================] - 0s 846us/step - loss: 3.6751e-04 - acc: 1.0000 - val_loss: 0.5778 - val_acc: 0.9060
Epoch 182/1000
435/435 [==============================] - 0s 938us/step - loss: 3.8304e-04 - acc: 1.0000 - val_loss: 0.5787 - val_acc: 0.9060
Epoch 183/1000
435/435 [==============================] - 0s 956us/step - loss: 3.5041e-04 - acc: 1.0000 - val_loss: 0.5812 - val_acc: 0.9056
Epoch 

Epoch 235/1000
435/435 [==============================] - 0s 846us/step - loss: 2.0436e-04 - acc: 1.0000 - val_loss: 0.6268 - val_acc: 0.9052
Epoch 236/1000
435/435 [==============================] - 0s 837us/step - loss: 2.0327e-04 - acc: 1.0000 - val_loss: 0.6311 - val_acc: 0.9048
Epoch 237/1000
435/435 [==============================] - 0s 828us/step - loss: 1.9297e-04 - acc: 1.0000 - val_loss: 0.6349 - val_acc: 0.9048
Epoch 238/1000
435/435 [==============================] - 0s 818us/step - loss: 1.9737e-04 - acc: 1.0000 - val_loss: 0.6357 - val_acc: 0.9044
Epoch 239/1000
435/435 [==============================] - 0s 837us/step - loss: 1.9826e-04 - acc: 1.0000 - val_loss: 0.6335 - val_acc: 0.9044
Epoch 240/1000
435/435 [==============================] - 0s 837us/step - loss: 1.9329e-04 - acc: 1.0000 - val_loss: 0.6349 - val_acc: 0.9037
Epoch 241/1000
435/435 [==============================] - 0s 846us/step - loss: 1.9959e-04 - acc: 1.0000 - val_loss: 0.6365 - val_acc: 0.9041
Epoch 

...

Epoch 756/1000
435/435 [==============================] - 0s 855us/step - loss: 3.6802e-06 - acc: 1.0000 - val_loss: 0.9642 - val_acc: 0.9021
Epoch 757/1000
435/435 [==============================] - 0s 864us/step - loss: 3.5863e-06 - acc: 1.0000 - val_loss: 0.9652 - val_acc: 0.9021
Epoch 758/1000
435/435 [==============================] - 0s 855us/step - loss: 3.3916e-06 - acc: 1.0000 - val_loss: 0.9622 - val_acc: 0.9025
Epoch 759/1000
435/435 [==============================] - 0s 864us/step - loss: 3.4658e-06 - acc: 1.0000 - val_loss: 0.9597 - val_acc: 0.9025
Epoch 760/1000
435/435 [==============================] - 0s 855us/step - loss: 3.4733e-06 - acc: 1.0000 - val_loss: 0.9581 - val_acc: 0.9025
Epoch 761/1000
435/435 [==============================] - 0s 855us/step - loss: 3.4570e-06 - acc: 1.0000 - val_loss: 0.9600 - val_acc: 0.9025
Epoch 762/1000
435/435 [==============================] - 0s 864us/step - loss: 3.5465e-06 - acc: 1.0000 - val_loss: 0.9598 - val_acc: 0.9025
Epoch 

Epoch 814/1000
435/435 [==============================] - 0s 910us/step - loss: 2.3649e-06 - acc: 1.0000 - val_loss: 0.9674 - val_acc: 0.9041
Epoch 815/1000
435/435 [==============================] - 0s 910us/step - loss: 2.4261e-06 - acc: 1.0000 - val_loss: 0.9690 - val_acc: 0.9041
Epoch 816/1000
435/435 [==============================] - 0s 910us/step - loss: 2.5246e-06 - acc: 1.0000 - val_loss: 0.9696 - val_acc: 0.9044
Epoch 817/1000
435/435 [==============================] - 0s 901us/step - loss: 2.4828e-06 - acc: 1.0000 - val_loss: 0.9709 - val_acc: 0.9041
Epoch 818/1000
435/435 [==============================] - 0s 892us/step - loss: 2.2961e-06 - acc: 1.0000 - val_loss: 0.9725 - val_acc: 0.9041
Epoch 819/1000
435/435 [==============================] - 0s 855us/step - loss: 2.2352e-06 - acc: 1.0000 - val_loss: 0.9766 - val_acc: 0.9041
Epoch 820/1000
435/435 [==============================] - 0s 846us/step - loss: 2.3010e-06 - acc: 1.0000 - val_loss: 0.9765 - val_acc: 0.9041
Epoch 

Epoch 872/1000
435/435 [==============================] - 0s 855us/step - loss: 1.6712e-06 - acc: 1.0000 - val_loss: 1.0248 - val_acc: 0.9017
Epoch 873/1000
435/435 [==============================] - 0s 846us/step - loss: 1.5607e-06 - acc: 1.0000 - val_loss: 1.0181 - val_acc: 0.9025
Epoch 874/1000
435/435 [==============================] - 0s 837us/step - loss: 1.6551e-06 - acc: 1.0000 - val_loss: 1.0149 - val_acc: 0.9025
Epoch 875/1000
435/435 [==============================] - 0s 855us/step - loss: 1.5929e-06 - acc: 1.0000 - val_loss: 1.0121 - val_acc: 0.9029
Epoch 876/1000
435/435 [==============================] - 0s 828us/step - loss: 1.5156e-06 - acc: 1.0000 - val_loss: 1.0111 - val_acc: 0.9033
Epoch 877/1000
435/435 [==============================] - 0s 846us/step - loss: 1.5111e-06 - acc: 1.0000 - val_loss: 1.0115 - val_acc: 0.9033
Epoch 878/1000
435/435 [==============================] - 0s 846us/step - loss: 1.4956e-06 - acc: 1.0000 - val_loss: 1.0127 - val_acc: 0.9033
Epoch 

Epoch 930/1000
435/435 [==============================] - 0s 966us/step - loss: 1.0751e-06 - acc: 1.0000 - val_loss: 1.0302 - val_acc: 0.9029
Epoch 931/1000
435/435 [==============================] - 0s 966us/step - loss: 1.1178e-06 - acc: 1.0000 - val_loss: 1.0298 - val_acc: 0.9033
Epoch 932/1000
435/435 [==============================] - 0s 910us/step - loss: 1.2152e-06 - acc: 1.0000 - val_loss: 1.0244 - val_acc: 0.9033
Epoch 933/1000
435/435 [==============================] - 0s 901us/step - loss: 1.0862e-06 - acc: 1.0000 - val_loss: 1.0202 - val_acc: 0.9041
Epoch 934/1000
435/435 [==============================] - 0s 956us/step - loss: 1.0352e-06 - acc: 1.0000 - val_loss: 1.0196 - val_acc: 0.9041
Epoch 935/1000
435/435 [==============================] - 0s 892us/step - loss: 1.1304e-06 - acc: 1.0000 - val_loss: 1.0206 - val_acc: 0.9041
Epoch 936/1000
435/435 [==============================] - 0s 920us/step - loss: 1.0871e-06 - acc: 1.0000 - val_loss: 1.0232 - val_acc: 0.9033
Epoch 

Epoch 988/1000
435/435 [==============================] - 0s 892us/step - loss: 7.7556e-07 - acc: 1.0000 - val_loss: 1.0412 - val_acc: 0.9033
Epoch 989/1000
435/435 [==============================] - 0s 901us/step - loss: 7.4277e-07 - acc: 1.0000 - val_loss: 1.0416 - val_acc: 0.9033
Epoch 990/1000
435/435 [==============================] - 0s 910us/step - loss: 7.1496e-07 - acc: 1.0000 - val_loss: 1.0422 - val_acc: 0.9041
Epoch 991/1000
435/435 [==============================] - 0s 892us/step - loss: 7.1652e-07 - acc: 1.0000 - val_loss: 1.0448 - val_acc: 0.9037
Epoch 992/1000
435/435 [==============================] - 0s 901us/step - loss: 7.2154e-07 - acc: 1.0000 - val_loss: 1.0448 - val_acc: 0.9037
Epoch 993/1000
435/435 [==============================] - 0s 883us/step - loss: 6.9715e-07 - acc: 1.0000 - val_loss: 1.0463 - val_acc: 0.9037
Epoch 994/1000
435/435 [==============================] - 0s 837us/step - loss: 7.1964e-07 - acc: 1.0000 - val_loss: 1.0472 - val_acc: 0.9037
Epoch 

In [14]:
model.save_weights('OTM_weights.h5')

In [15]:
#Reading the data with missing Categories.
nocat = pd.read_excel("NoCat.xlsx")
nocat

,Category,Status,Company,Contact Person,Designation,Decision maker,Email ID,Phone,Notes prospects (for sentiment analisys),Country,Website,Address,Notes sales process,Unnamed: 13,Unnamed: 14,Unnamed: 15
577,NaN,Cold lead,CRB CITYRAMA Travel Agency,Tünde MOGYORÓSI,Manager of Incoming Department,yes,Cityrama@cityrama.hu,NaN,NaN,Hungary,NaN,NaN,"If I have to hit a button only, I would hit: 4...",NaN,NaN,NaN
588,NaN,NaN,HOTEL INTERNATIONAL IASI,AlexandruHOLBANEL,Sales Executive,yes,sales@hotelinternationaliasi.ro,NaN,NaN,Romania,NaN,NaN,NaN,NaN,NaN,NaN
589,NaN,NaN,Tirolwerbung,AngelaSemrajc,Employee,yes,Angela.semrajc@tirolwerbung.at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
591,NaN,NaN,Polish National Tourist Office,AnnaCichońska,Director,yes,NaN,NaN,USA and Canda market,Poland,NaN,NaN,NaN,NaN,NaN,NaN
592,NaN,NaN,Rynek Turystyczny,AnnaJanuszewska,Marketing and Commercials Specialist,yes,a.januszewska@pws-promedia.pl,NaN,This is Tourist Magazine - partner with Polski...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593,NaN,NaN,Zadar Tourist Board,AnteGalić,Public Relations & Marketing,yes,ante@zadar.travel,NaN,NaN,Croatia,NaN,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,Biuro Wadi,ArturPiszczek,Owner,yes,biuro@wadi.pl,NaN,NaN,Poland,NaN,NaN,NaN,NaN,NaN,NaN
595,NaN,NaN,Emilia Kubik,BarbaraTutak,"Główny Specjalista, Lider Projektu",yes,barbara.tutak@pot.gov.pl,NaN,NaN,Poland,NaN,NaN,NaN,NaN,NaN,NaN
596,NaN,NaN,Orthanna Spltd,BoyanaKostadinova,Manager,yes,b.kostadinova@orthanna.com,NaN,NaN,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN
597,NaN,NaN,88 Rooms,BranimirMladenovic,Sales and marketing administrator,yes,branimir.mladenovic@88rooms.com,NaN,NaN,Serbia,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#Splitting the 'Company column'
nc = nocat.loc[0:899,['Company']]

In [17]:
nc

,Company
577,CRB CITYRAMA Travel Agency
588,HOTEL INTERNATIONAL IASI
589,Tirolwerbung
591,Polish National Tourist Office
592,Rynek Turystyczny
593,Zadar Tourist Board
594,Biuro Wadi
595,Emilia Kubik
596,Orthanna Spltd
597,88 Rooms


In [18]:
#Applying the same cleaning techniques applied above.
nc['Company'] = nc['Company'].str.replace('[{}]'.format(string.punctuation), '')
nc['Company'] = nc['Company'].str.replace('-', '')
nc['Company'] = nc['Company'].str.replace('.', '')
nc['Company'] = nc['Company'].str.replace(' - ', '')
nc

,Company
577,CRB CITYRAMA Travel Agency
588,HOTEL INTERNATIONAL IASI
589,Tirolwerbung
591,Polish National Tourist Office
592,Rynek Turystyczny
593,Zadar Tourist Board
594,Biuro Wadi
595,Emilia Kubik
596,Orthanna Spltd
597,88 Rooms


In [19]:
# Creating the sequence
vocabulary_size = 5000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(nc['Company'])
sequences = tokenizer.texts_to_sequences(nc['Company'])
d = pad_sequences(sequences, maxlen=6)

In [20]:
#Predicting the Categories for the Companies.
model.predict(d, verbose = 1)

112/112 [==============================] - 0s 1ms/step


array([[3.00115417e-08, 6.95685500e-08, 2.00053429e-12, ...,
        6.47513065e-09, 1.01775306e-07, 9.99999523e-01],
       [1.59667586e-08, 1.00082238e-04, 1.01152928e-07, ...,
        3.27733233e-07, 8.38580494e-09, 5.91121420e-07],
       [3.19077631e-09, 3.02906251e-08, 3.82362926e-11, ...,
        1.63523362e-10, 1.07496562e-04, 1.96106527e-02],
       ...,
       [3.03153480e-09, 1.07235870e-07, 2.71136950e-11, ...,
        3.50680907e-09, 5.08386302e-06, 1.28572639e-02],
       [6.70873533e-07, 6.64732411e-08, 2.13784777e-04, ...,
        1.02658196e-04, 9.95112598e-01, 1.40551226e-09],
       [4.96869967e-09, 7.21711269e-09, 3.49372332e-13, ...,
        1.18073089e-08, 9.06152081e-07, 9.99329329e-01]], dtype=float32)

In [21]:
prediction = model.predict(d)

In [22]:
prediction

array([[3.00115417e-08, 6.95685500e-08, 2.00053429e-12, ...,
        6.47513065e-09, 1.01775306e-07, 9.99999523e-01],
       [1.59667586e-08, 1.00082238e-04, 1.01152928e-07, ...,
        3.27733233e-07, 8.38580494e-09, 5.91121420e-07],
       [3.19077631e-09, 3.02906251e-08, 3.82362926e-11, ...,
        1.63523362e-10, 1.07496562e-04, 1.96106527e-02],
       ...,
       [3.03153480e-09, 1.07235870e-07, 2.71136950e-11, ...,
        3.50680907e-09, 5.08386302e-06, 1.28572639e-02],
       [6.70873533e-07, 6.64732411e-08, 2.13784777e-04, ...,
        1.02658196e-04, 9.95112598e-01, 1.40551226e-09],
       [4.96869967e-09, 7.21711269e-09, 3.49372332e-13, ...,
        1.18073089e-08, 9.06152081e-07, 9.99329329e-01]], dtype=float32)

In [23]:
prediction[1]

array([1.5966759e-08, 1.0008224e-04, 1.0115293e-07, 5.9195918e-07,
       1.1491292e-10, 9.9790323e-01, 2.9122580e-07, 1.4314303e-07,
       3.2773323e-07, 8.3858049e-09, 5.9112142e-07], dtype=float32)

In [24]:
p =(prediction == prediction.max(axis=1)[:,None]).astype(float)

In [25]:
p[5]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [26]:
#This is how the prediction looks like in a dataframe.
df = pd.DataFrame(p)
df

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
df['Category'] = 'x'
df

,0,1,2,3,4,5,6,7,8,9,10,Category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,x
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,x
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,x
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,x
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,x
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,x
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,x
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,x
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,x
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,x


In [28]:
#Decoding the onehot-encoded values to give a readable output
df.loc[df[0] == 1.0, 'Category'] = 'A'
df.loc[df[1] == 1.0, 'Category'] = 'CB'
df.loc[df[2] == 1.0, 'Category'] = 'CC'
df.loc[df[3] == 1.0, 'Category'] = 'CH'
df.loc[df[4] == 1.0, 'Category'] = 'EP'
df.loc[df[5] == 1.0, 'Category'] = 'H'
df.loc[df[6] == 1.0, 'Category'] = 'R'
df.loc[df[7] == 1.0, 'Category'] = 'TA'
df.loc[df[8] == 1.0, 'Category'] = 'TB'
df.loc[df[9] == 1.0, 'Category'] = 'TO'
df.loc[df[10] == 1.0, 'Category'] = 'WP'

In [29]:
df

,0,1,2,3,4,5,6,7,8,9,10,Category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,WP
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,H
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,WP
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,WP
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,WP
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CB
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,WP
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,TO
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,TO
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,TO


In [30]:
x = nocat['Company']
x.index = range(112)
x = pd.DataFrame(x)
x

,Company
0,CRB CITYRAMA Travel Agency
1,HOTEL INTERNATIONAL IASI
2,Tirolwerbung
3,Polish National Tourist Office
4,Rynek Turystyczny
5,Zadar Tourist Board
6,Biuro Wadi
7,Emilia Kubik
8,Orthanna Spltd
9,88 Rooms


In [31]:
df = pd.concat([df, x['Company']], axis = 1)

In [32]:
#Final output for the UnCategorised data.
df = df[['Category', 'Company']]
df

,Category,Company
0,WP,CRB CITYRAMA Travel Agency
1,H,HOTEL INTERNATIONAL IASI
2,WP,Tirolwerbung
3,WP,Polish National Tourist Office
4,WP,Rynek Turystyczny
5,CB,Zadar Tourist Board
6,WP,Biuro Wadi
7,TO,Emilia Kubik
8,TO,Orthanna Spltd
9,TO,88 Rooms


In [33]:
#Saving it to an Excel sheet
df.to_excel("pred.xlsx")

In [37]:
#Over here you can see that the number of Wedding Planner(WP) and Hotel(H) is significantly higher than the rest of the categories.
#This in-turn causes the model to be biased mainly towards WP and aslo a little towards H.
ldata['Category'].value_counts()

WP    228
H     124
TO     70
TB     62
CB     57
TA     49
CH     35
R      19
EP     18
A       4
CC      4
Name: Category, dtype: int64

In [38]:
# Here you can see that the number of Wedding planer(WP) in the prediction seems to be significantly high.
# Though the model is trained at an accuracy of ~90%, the model is biased due to the very high amount of the WP elements in the training Input.
df['Category'].value_counts()

WP    50
TO    23
H     17
CB    12
EP     3
CH     3
TB     2
TA     2
Name: Category, dtype: int64